In [1]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd
import GOSTRocks.rasterMisc as rMisc

from shapely.geometry import Point, LineString

sys.path.append("../src")

import GOST_AIS.ais_data_helper as ais

In [ ]:
importlib.reload(ais)

in_file = 's3://wbgdecinternal-ntl/AIS_TMP/MMSI_247298800.csv'
inD = ais.ais_file(in_file)
tempD = inD.read_simple_geom()
tempD['DAY'] = tempD['timestamp'].apply(lambda x: x[:10])

In [ ]:
tempD.to_file(f'/home/wb411133/temp/AIS/{os.path.basename(in_file).replace(".csv","")}.geojson', driver="GeoJSON")

In [ ]:
# Generate lines file for individual ship data
importlib.reload(ais)
line_features = ais.generate_linear_features(tempD)
line_features.to_file(f'/home/wb411133/temp/AIS/{os.path.basename(in_file).replace(".csv","_lines_simple")}.geojson', driver="GeoJSON")

# Processing daily files

In [2]:
daily_file = 's3://wbgdecinternal-ntl/AIS_TMP/MAY2.csv'
inD = pd.read_csv(daily_file)

In [4]:
inD = inD.loc[~inD['latitude'].isna()]
inD['DAY'] = inD['timestamp'].apply(lambda x: x[:10])

In [5]:
inD['DAY'].value_counts()

2020-05-02    4826642
2019-05-02          1
Name: DAY, dtype: int64

In [6]:
inD_geom = [Point(x['longitude'], x['latitude']) for idx, x in inD.iterrows()]
inGPD = gpd.GeoDataFrame(inD, geometry=inD_geom, crs='epsg:4326')

In [7]:
rMisc.rasterizeDataFrame?

In [9]:
out_file = "/home/wb411133/temp/AIS/may2.tif"
rMisc.rasterizeDataFrame(inGPD, out_file, res=0.1, mergeAlg="ADD")

{'init': 'epsg:4326'}


{'meta': {'count': 1,
  'crs': {'init': 'epsg:4326'},
  'dtype': dtype('int16'),
  'driver': 'GTiff',
  'transform': Affine(0.09999992592592592, 0.0, -179.99984,
         0.0, -0.09997502414113278, 89.52888),
  'height': 1795,
  'width': 3600},
 'vals': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16)}